In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, activations
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import mean_absolute_error

from tinymlgen import port

import numpy as np
from numpy import asarray

from utils.preprocessing import preprocessImage
from utils.grad_cam import get_img_array, make_gradcam_heatmap, save_and_display_gradcam
from utils.image_display import imageDisplay
from utils.transform_data import transformData

In [2]:
#Preprocessing
resize_image = True
#new_size = (480, 640)
new_size = (640, 480)
grey_scale = True
rgb_or_grey = 3
normalize_image = False
thresholding = False
erosion = False
kernel = np.ones((2, 2), np.uint8)
cut_image = False
turn_image = False

#Scaling
min_max = False
z_score = False

min_max_labels = True
z_score_labels = False

#Training
batch_size = 16
epochs = 150

if resize_image == True:
    #new_size = (300, 225)
    #new_size = (225, 300)
    if turn_image == True:
        #new_size = (160, 120)
        new_size = (120, 160)
        #new_size = (240, 320)
    else:
        new_size = (160, 120)
        #new_size = (320, 240)

if grey_scale == True:
    rgb_or_grey = 1

preprocessImage = preprocessImage()

In [3]:
imageFoldersValidation = "../Bilder/Manometer/Blindset final test"

#Load in Images and Clean DataFrame
validation_images, validation_labels = preprocessImage.load_images(imageFoldersValidation, True, grey_scale, resize_image, new_size, normalize_image, thresholding, erosion, kernel, cut_image, turn_image)

validation_images = validation_images.reshape(len(validation_labels), new_size[1], new_size[0], rgb_or_grey)
nRows,nCols,nDims = validation_images.shape[1:]
validation_images = validation_images.reshape(validation_images.shape[0], nRows, nCols, nDims)
#Scale Data
transformer = transformData()

#Parameters Training data
mean_value = 114.31817292906746
standard_deviation = 77.30169303183406

min_value = 0
max_value = 255

#Parameters Targetr Data
mean_value_labels = 5.035714285714286
standard_deviation_labels = 3.0150275548430043

min_labels = 0
max_labels = 10


if min_max == True:
    validation_images = transformer.scale_min_max(validation_images, min_value, max_value)

if z_score == True:
    validation_images = transformer.scale_z_score(validation_images, mean_value, standard_deviation)


if min_max_labels == True:
    validation_labels = transformer.scale_min_max(validation_labels, min_labels, max_labels)

if z_score_labels == True:
    validation_labels = transformer.scale_z_score(validation_labels, mean_value_labels, standard_deviation_labels)


imageFolderPath: ../Bilder/Manometer/Blindset final test


In [4]:
image_num = len(validation_labels)-1

path = "models/model5/model.tflite"

from sklearn.metrics import mean_squared_error

preds = []

interpreter = tf.lite.Interpreter(model_path=path)
interpreter.allocate_tensors()

for element in range(len(validation_labels)):
    
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_data = validation_images[element].reshape(1, 120, 160, 1)#.flatten()
    input_data = np.float32(input_data)
    interpreter.set_tensor(input_details[0]['index'], input_data)

    predictions=interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    output_data = output_data[0] * 10

    preds.append(output_data)

dict = {"True Value":"Predction"}
for A, B in zip(validation_labels*10, preds):
    dict[A] = B[0]

dict


{'True Value': 'Predction',
 0.3: 4.1880345,
 0.5: 6.8313284,
 0.8: 6.8313284,
 0.0: 2.106305,
 1.3: 14.37147,
 1.5: 16.167542,
 1.7999999999999998: 21.078522,
 1.0: 8.357575,
 10.0: 102.08226,
 2.3: 26.1178,
 2.5: 24.090656,
 2.8: 29.253881,
 2.0: 23.587637,
 3.3: 34.911182,
 3.5: 36.682953,
 3.8: 37.923553,
 3.0: 32.31759,
 4.3: 43.66263,
 4.5: 43.44719,
 4.8: 47.558037,
 4.0: 41.90794,
 5.300000000000001: 50.897453,
 5.5: 58.312485,
 5.8: 59.92215,
 5.0: 46.905434,
 6.3: 66.209274,
 6.5: 68.258606,
 6.799999999999999: 69.57468,
 6.0: 62.216537,
 7.3: 70.7912,
 7.5: 77.76207,
 7.800000000000001: 77.188065,
 7.0: 69.38487,
 8.3: 82.68796,
 8.5: 87.390594,
 8.8: 91.32857,
 8.0: 81.2138,
 9.3: 95.56228,
 9.5: 102.27203,
 9.8: 105.03544,
 9.0: 92.93342}

In [5]:
errors = mean_absolute_error(validation_labels, preds)
errors

51.188349273728164